In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from IPython.display import clear_output
from matplotlib.pyplot import imshow
import pandas as pd
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.metrics import Accuracy, Precision, Recall, MeanSquaredError, MeanAbsoluteError, AUC
from tensorflow.keras.layers import Input, Reshape, GRU, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.metrics import *
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import backend as k
import datetime
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report

In [ ]:
train_df = pd.read_csv("../input/ucf101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist01.txt",sep = " ",header = None,names = ['path','class'])

In [ ]:
content = []
i = 0
n = 50
while len(content) < 10 and i < 102:
    if len(train_df[train_df['class']==i]) >= n:
        df_temp = train_df[train_df['class']==i].iloc[:n]
        path = df_temp['path']
        content.append(path)
        
    i += 1
    

content = np.array(content)

In [ ]:
content = content.reshape(500,)
y = np.array([i//50 for i in range(0,500)])

In [ ]:
def read_frames(root_folder,arr,each_nth=10):
    videos=[]
    for j  in range(len(arr)):
        clear_output()
        print(np.round(100*j/len(arr),3))
            
        vcap=cv2.VideoCapture(root_folder+arr[j])
        success=True
  
        frames=[]
        cnt=0
        while success:
            try:
              success,image=vcap.read()
              cnt+=1
              if cnt%each_nth==0:
                image=resize(image,(128,192))
                frames.append(image)
            except Exception as e:
                print(e)
        videos.append(frames)
    
    return videos

In [ ]:
def select_frames(frames_arr , n=10):
    videos=[]
    for i in range(len(frames_arr)):
        frames=[]
        for t in np.linspace(0, len(frames_arr[i])-1, num=n):
            frames.append(frames_arr[i][int(t)])
        videos.append(frames)
        
    videos = np.array(videos)
    print(videos.shape)
    return videos

In [ ]:
X_frames = read_frames("../input/ucf101/UCF101/UCF-101/",content)
X_frames = select_frames(X_frames, 10)

In [ ]:
xtr, xte, ytr , yte = train_test_split(X_frames , y , shuffle=True,test_size = 0.2,random_state = 42)

In [ ]:
ytr = to_categorical(ytr, 10)
yte = to_categorical(yte,10)

# **Random Sample Visualization**

In [ ]:
fig = plt.figure(figsize=(32,8))

random_video_index = np.random.randint(0,len(X_frames))

for i,image in enumerate(X_frames[random_video_index]):
    ax = plt.subplot(2,5,i+1)
    imshow(image)

In [ ]:
def list_to_stack(xs):
  xs=tf.stack(xs, axis=1)
  s = tf.shape(xs)
 
  return xs

# Hybrid Model

In [ ]:
ish=(10, 128, 192, 3)
  
xs=[]


inp = Input(ish)

for slice_indx in range(0,10,1):
  x=Lambda(lambda x: x[:, slice_indx])(inp)
  x=BatchNormalization(momentum=0.8)(x)
  x=Conv2D(filters=20, kernel_size=3, padding='same', activation='relu')(x)
  x=BatchNormalization(momentum=0.8)(x)
  x=MaxPooling2D(pool_size=2)(x)
  
  x=Conv2D(filters=30, kernel_size=3, padding='same', activation='relu')(x)
  x=BatchNormalization(momentum=0.8)(x)
  x=MaxPooling2D(pool_size=2)(x)
  x=Conv2D(filters=30, kernel_size=3, padding='same', activation='relu')(x)
    
  xs.append(x)
  

t=Lambda(list_to_stack)(xs)
t=Conv3D(50,3,padding='same')(t)
t=BatchNormalization(momentum=0.8)(t)
target_shape=(10,32*48*50)
t=Reshape(target_shape)(t)
t=GRU(25, return_sequences=True)(t)
t=GRU(50, return_sequences=False,dropout=0.5)(t)

t=Dense(100,'relu')(t)
out=Dense(10, activation='softmax')(t)

model = Model(inputs=inp, outputs=out)

# Optimizers

In [ ]:
opt = SGD(lr=0.0087)
adam_optimizer = Adam(learning_rate=0.001)
rmsprop_optimizer = RMSprop(learning_rate=0.001)

# Compile Model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=[
        'accuracy',
        Precision(),
        Recall(),
        MeanSquaredError(),
        MeanAbsoluteError(),
        AUC(),
    ]
)

model.summary()

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
hibrid_model = model.fit(xtr, ytr, epochs=25, batch_size=10,validation_data=(xte, yte),shuffle  = True)

In [ ]:
latest_metrics = {key: value[-1] for key, value in hibrid_model.history.items()}
metrics_table = pd.DataFrame(list(latest_metrics.items()), columns=['Metric', 'Latest Value'])
print(metrics_table)

In [ ]:
y_pred = model.predict(xte)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(yte, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

# Model performance Visualization

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(hibrid_model.history['loss'], label='Training loss')
ax.legend(loc="upper left")
plt.ylabel('Loss')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(hibrid_model.history['val_loss'], label='Vallidation loss')
ax.legend(loc="upper left")
fig.suptitle('Model loss')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(hibrid_model.history['accuracy'], label='Training accuracy')
ax.legend(loc="upper left")
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(hibrid_model.history['val_accuracy'], label='Vallidation accuracy')
ax.legend(loc="upper left")
fig.suptitle('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
plt.show()

# Adam Optimizer

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = adam_optimizer, metrics=[
        'accuracy',
        Precision(),
        Recall(),
        MeanSquaredError(),
        MeanAbsoluteError(),
        AUC(),
    ])
model.summary()

In [ ]:
hibrid_model_adamOpt = model.fit(xtr, ytr, epochs=25, batch_size=10,validation_data=(xte, yte),shuffle  = True)

In [ ]:
latest_metrics = {key: value[-1] for key, value in hibrid_model_adamOpt.history.items()}
metrics_table = pd.DataFrame(list(latest_metrics.items()), columns=['Metric', 'Latest Value'])
print(metrics_table)

In [ ]:
y_pred = model.predict(xte)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(yte, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

# Performance Visualization

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(hibrid_model_adamOpt.history['loss'], label='Training loss')
ax.legend(loc="upper left")
plt.ylabel('Loss')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(hibrid_model_adamOpt.history['val_loss'], label='Vallidation loss')
ax.legend(loc="upper left")
fig.suptitle('Model loss')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(hibrid_model_adamOpt.history['accuracy'], label='Training accuracy')
ax.legend(loc="upper left")
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(hibrid_model_adamOpt.history['val_accuracy'], label='Vallidation accuracy')
ax.legend(loc="upper left")
fig.suptitle('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
plt.show()

# RmsProp Optimizer

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = rmsprop_optimizer, metrics=[
        'accuracy',
        Precision(),
        Recall(),
        MeanSquaredError(),
        MeanAbsoluteError(),
        AUC(),
    ])
model.summary()

In [ ]:
hybrid_model_RMSProp = model.fit(xtr, ytr, epochs=25, batch_size=10,validation_data=(xte, yte),shuffle  = True)

In [ ]:
latest_metrics = {key: value[-1] for key, value in history_3.history.items()}
metrics_table = pd.DataFrame(list(latest_metrics.items()), columns=['Metric', 'Latest Value'])
print(metrics_table)

# Permormance Visualization

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(history_3.history['loss'], label='Training loss')
ax.legend(loc="upper left")
plt.ylabel('Loss')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(history_3.history['val_loss'], label='Vallidation loss')
ax.legend(loc="upper left")
fig.suptitle('Model loss')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(history_3.history['accuracy'], label='Training accuracy')
ax.legend(loc="upper left")
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(history_3.history['val_accuracy'], label='Vallidation accuracy')
ax.legend(loc="upper left")
fig.suptitle('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
plt.show()

# Model demonstration

In [ ]:
classes = []
for i in range(len(content)):
    class_name = content[i].split('/')[0]
    if class_name not in classes:
        classes.append(class_name)
        
num_classes = [i for i in range(10)]
class_map = dict(zip(num_classes,classes))

In [ ]:
random_videos = np.random.randint(0,len(xte),size=(10))
predicted  = model.predict(xte[random_videos],batch_size = 10)
predicted  = np.argmax(predicted,axis=1)

In [ ]:
fig = plt.figure(figsize=(30,10))

for i,rand_indx in enumerate(random_videos):
    ax = plt.subplot(2,5,i+1)
    video = xte[rand_indx]
    frame = video[np.random.randint(0,10)]
    ax.set_title(class_map[predicted[i]])
    imshow(frame)

# CNN Model

In [ ]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, BatchNormalization, Flatten, Dense, Reshape, GRU
from tensorflow.keras.models import Model

# Define input shape for the 3D CNN
input_shape = (10, 128, 192, 3)  # Assuming 3D data with dimensions (time, height, width, channels)

# Create the input layer
inp = Input(shape=input_shape)

# 3D Convolutional layers
x = Conv3D(20, (3, 3, 3), padding='same', activation='relu')(inp)
x = BatchNormalization(momentum=0.8)(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)

x = Conv3D(30, (3, 3, 3), padding='same', activation='relu')(x)
x = BatchNormalization(momentum=0.8)(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)

# Flatten the output for further processing
x = Flatten()(x)

num_classes = 10
out = Dense(num_classes, activation='softmax')(x)

# Create the CNN model
cnn_model = Model(inputs=inp, outputs=out)

# Display the CNN model summary
cnn_model.summary()

In [ ]:
plot_model(cnn_model, to_file='cnn_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[
        'accuracy',
        Precision(),
        Recall(),
        MeanSquaredError(),
        MeanAbsoluteError(),
        AUC(),
    ])

In [ ]:
cnn_Model_history = cnn_model.fit(xtr, ytr, epochs=25, batch_size=10,validation_data=(xte, yte),shuffle  = True)

In [ ]:
latest_metrics = {key: value[-1] for key, value in cnn_Model_history.history.items()}
metrics_table = pd.DataFrame(list(latest_metrics.items()), columns=['Metric', 'Latest Value'])
print(metrics_table)

In [ ]:
y_pred = cnn_model.predict(xte)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(yte, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

# Performance Visualization

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(cnn_Model_history.history['loss'], label='Training loss')
ax.legend(loc="upper left")
plt.ylabel('Loss')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(cnn_Model_history.history['val_loss'], label='Vallidation loss')
ax.legend(loc="upper left")
fig.suptitle('Model loss')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(cnn_Model_history.history['accuracy'], label='Training accuracy')
ax.legend(loc="upper left")
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(cnn_Model_history.history['val_accuracy'], label='Vallidation accuracy')
ax.legend(loc="upper left")
fig.suptitle('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
plt.show()

# Model demonstration

In [ ]:
random_videos = np.random.randint(0,len(xte),size=(10))
predicted  = cnn_model.predict(xte[random_videos],batch_size = 10)
predicted  = np.argmax(predicted,axis=1)

In [ ]:
fig = plt.figure(figsize=(30,10))

for i,rand_indx in enumerate(random_videos):
    ax = plt.subplot(2,5,i+1)
    video = xte[rand_indx]
    frame = video[np.random.randint(0,10)]
    ax.set_title(class_map[predicted[i]])
    imshow(frame)

# RNN Model

In [ ]:
input_shape = (10, 128, 192, 3)

inp = Input(shape=input_shape)

reshaped_input = Reshape((10, -1))(inp)

rnn = GRU(64, return_sequences=True)(reshaped_input)
rnn_output = GRU(64)(rnn)

num_classes = 10
out = Dense(num_classes, activation='softmax')(rnn_output)

rnn_model = Model(inputs=inp, outputs=out)

rnn_model.summary()

In [ ]:
rnn_model.compile(optimizer=adam_optimizer, loss=loss, metrics=[
        'accuracy',
        Precision(),
        Recall(),
        MeanSquaredError(),
        MeanAbsoluteError(),
        AUC(),
    ])

In [ ]:
plot_model(rnn_model, to_file='rnn_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
rnn_Model_history = rnn_model.fit(xtr, ytr, epochs=25, batch_size=10,validation_data=(xte, yte),shuffle  = True)

In [ ]:
latest_metrics = {key: value[-1] for key, value in rnn_Model_history.history.items()}
metrics_table = pd.DataFrame(list(latest_metrics.items()), columns=['Metric', 'Latest Value'])
print(metrics_table)

In [ ]:
y_pred = rnn_model.predict(xte)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(yte, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

# Performance Visualization

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(rnn_Model_history.history['loss'], label='Training loss')
ax.legend(loc="upper left")
plt.ylabel('Loss')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(rnn_Model_history.history['val_loss'], label='Vallidation loss')
ax.legend(loc="upper left")
fig.suptitle('Model loss')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,6))

ax = plt.subplot(1,2,1)
plt.plot(rnn_Model_history.history['accuracy'], label='Training accuracy')
ax.legend(loc="upper left")
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
ax = plt.subplot(1,2,2)
plt.plot(rnn_Model_history.history['val_accuracy'], label='Vallidation accuracy')
ax.legend(loc="upper left")
fig.suptitle('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
plt.show()